In [2]:
from création_data import param_Dataset
import fonction_model as f
from IPython.display import display, Audio
from scipy.io.wavfile import write
import pickle


In [3]:
Fe = 2**15

## Medium 1:

In [4]:
Fichier = "Data_set_acc_1/plexi/"
N_sample, param_dataset, xinB = param_Dataset(N_sample=10,
                                             article_C = False,
                                             acier_1C = True,
                                             acier_2C = False, 
                                             medium_1T = False, 
                                             medium_2T = False, 
                                             metal_T = False,
                                             plexi_T = True)

e:  207900000000.0 212100000000.0
i:  9.274548588504176e-15 1.0047025338089098e-14
b:  0.0019281786515500182 0.0021309740742086977


In [ ]:
T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly = param_dataset[0]
Q = f.Main(T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly,xinB,Fe) 

In [ ]:
display(Audio(Q,rate=Fe))

In [ ]:
from scipy.io import wavfile, loadmat
C1P1mat = loadmat("../../Mesures/Mesures_2023.01.27_LAM_normalise/Medium_1/Corde_Acier/C_2_P_1.mat")
acc = C1P1mat["acc_t"].reshape(-1)
fs2 = int(C1P1mat["fs"].reshape(-1))

In [ ]:
display(Audio(acc,rate = fs2))

In [ ]:
import numpy as np
Nfft = len(acc)
Y2 = np.fft.rfft(acc, Nfft)
fy2 = np.fft.rfftfreq(Nfft,1/fs2)

In [ ]:

Q_f = np.fft.rfft(Q)
#F_f = np.fft.rfft(F)
#F_f2 = np.fft.rfft(F_2)

freq = np.fft.rfftfreq(len(F)) * Fe

import matplotlib.pyplot as plt
%matplotlib widget

plt.figure()
plt.semilogx(freq,20*np.log10(np.abs(Q_f)),label='accel')
#plt.semilogx(freq,20*np.log10(np.abs(F_f)/np.max(np.abs(F_f))),label='Force_corde')
#plt.semilogx(freq,20*np.log10(np.abs(F_f2)/np.max(np.abs(F_f2))),label='Force_table')
#plt.semilogx(freq,20*np.log10(np.abs(F_f2 / F_f)),label='Force dif')

plt.semilogx(fy2, 20*np.log10(np.abs(Y2)),label='Mesure')

plt.xlim(20,3000)
plt.legend()
plt.grid()
plt.show()

In [ ]:
display(Audio(Q,rate=Fe))

In [ ]:
for i in range(N_sample):
    T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly = param_dataset[i]
    F = f.Main(T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly,xinB,Fe) 
    #display(Audio(F,rate=Fe))  
    write(Fichier + str(i) + '.wav',rate=Fe ,data = F / max(F))

In [ ]:
dico = {}
for i in range(N_sample):
    kle = str(i) + ".wav"
    dico[kle] = {}
    dico[kle]['T_corde'] = param_dataset[i][0]
    dico[kle]['rho_l_corde'] = param_dataset[i][1]
    dico[kle]['L_corde'] = param_dataset[i][2]
    dico[kle]['E_corde'] = param_dataset[i][3]
    dico[kle]['I_corde'] = param_dataset[i][4]
    dico[kle]['h_table'] = param_dataset[i][5]
    dico[kle]['E_nu_table'] = param_dataset[i][6]
    dico[kle]['rho_table'] = param_dataset[i][7]
    dico[kle]['Lx_table'] = param_dataset[i][8]
    dico[kle]['Ly_table'] = param_dataset[i][9]

with open(Fichier + 'parametres.pickle', 'wb') as f:
    pickle.dump(dico, f)


## medium 2

In [ ]:
Fichier = "1ier_data_set/medium_2/"
N_sample, param_dataset, xinB = param_Dataset(N_sample=200,
                                             article_C = True,
                                             acier_1C = False,
                                             acier_2C = False, 
                                             medium_1T = False, 
                                             medium_2T = True, 
                                             metal_T = False,
                                             plexi_T = False)

In [ ]:
for i in range(N_sample):
    T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly = param_dataset[i]
    F = f.Main(T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly,xinB,Fe)   
    write(Fichier + str(i) + '.wav',rate=Fe ,data = F / max(F))

In [ ]:
dico = {}
for i in range(N_sample):
    kle = str(i) + ".wav"
    dico[kle] = {}
    dico[kle]['T_corde'] = param_dataset[i][0]
    dico[kle]['rho_l_corde'] = param_dataset[i][1]
    dico[kle]['L_corde'] = param_dataset[i][2]
    dico[kle]['E_corde'] = param_dataset[i][3]
    dico[kle]['I_corde'] = param_dataset[i][4]
    dico[kle]['h_table'] = param_dataset[i][5]
    dico[kle]['E_nu_table'] = param_dataset[i][6]
    dico[kle]['rho_table'] = param_dataset[i][7]
    dico[kle]['Lx_table'] = param_dataset[i][8]
    dico[kle]['Ly_table'] = param_dataset[i][9]

with open(Fichier + 'parametres.pickle', 'wb') as f:
    pickle.dump(dico, f)


## Acier : 

In [ ]:
Fichier = "1ier_data_set/acier/"
N_sample, param_dataset, xinB = param_Dataset(N_sample=200,
                                             article_C = True,
                                             acier_1C = False,
                                             acier_2C = False, 
                                             medium_1T = False, 
                                             medium_2T = False, 
                                             metal_T = True,
                                             plexi_T = False)

In [ ]:
for i in range(N_sample):
    T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly = param_dataset[i]
    F = f.Main(T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly,xinB,Fe)  
    #display(Audio(F,rate=Fe)) 
    write(Fichier + str(i) + '.wav',rate=Fe ,data = F / max(F))

In [ ]:
dico = {}
for i in range(N_sample):
    kle = str(i) + ".wav"
    dico[kle] = {}
    dico[kle]['T_corde'] = param_dataset[i][0]
    dico[kle]['rho_l_corde'] = param_dataset[i][1]
    dico[kle]['L_corde'] = param_dataset[i][2]
    dico[kle]['E_corde'] = param_dataset[i][3]
    dico[kle]['I_corde'] = param_dataset[i][4]
    dico[kle]['h_table'] = param_dataset[i][5]
    dico[kle]['E_nu_table'] = param_dataset[i][6]
    dico[kle]['rho_table'] = param_dataset[i][7]
    dico[kle]['Lx_table'] = param_dataset[i][8]
    dico[kle]['Ly_table'] = param_dataset[i][9]

with open(Fichier + 'parametres.pickle', 'wb') as f:
    pickle.dump(dico, f)


## Plexi : 

In [ ]:
Fichier = "1ier_data_set/plexi/"
N_sample, param_dataset, xinB = param_Dataset(N_sample=200,
                                             article_C = True,
                                             acier_1C = False,
                                             acier_2C = False, 
                                             medium_1T = False, 
                                             medium_2T = False, 
                                             metal_T = False,
                                             plexi_T = True)

In [ ]:
for i in range(N_sample):
    T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly = param_dataset[i]
    F = f.Main(T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly,xinB,Fe)   
    write(Fichier + str(i) + '.wav',rate=Fe ,data = F / max(F))

In [ ]:
dico = {}
for i in range(N_sample):
    kle = str(i) + ".wav"
    dico[kle] = {}
    dico[kle]['T_corde'] = param_dataset[i][0]
    dico[kle]['rho_l_corde'] = param_dataset[i][1]
    dico[kle]['L_corde'] = param_dataset[i][2]
    dico[kle]['E_corde'] = param_dataset[i][3]
    dico[kle]['I_corde'] = param_dataset[i][4]
    dico[kle]['h_table'] = param_dataset[i][5]
    dico[kle]['E_nu_table'] = param_dataset[i][6]
    dico[kle]['rho_table'] = param_dataset[i][7]
    dico[kle]['Lx_table'] = param_dataset[i][8]
    dico[kle]['Ly_table'] = param_dataset[i][9]

with open(Fichier + 'parametres.pickle', 'wb') as f:
    pickle.dump(dico, f)
